# PCA(주성분분석)를 활용한 분류 노트북

이 Jupyter Notebook은 `scikit-learn`의 `load_digits` 데이터셋을 사용하여 **PCA(주성분분석)**를 통한 차원 축소의 효과를 보여주고, 원본 데이터, 스케일링된 데이터, 그리고 PCA가 적용된 데이터 각각에 대해 로지스틱 회귀 모델의 성능을 비교합니다.

**PCA(주성분분석) 개요:**
* **차원 축소**: 특성이 너무 많을 때 각 특성들로부터 새로운 특성을 만들어냅니다.
* **다중공선성 처리**: 특성들 간의 상호 연관관계가 있을 때, PCA는 이러한 부분들까지 고려하여 새로운 요소를 만듭니다.
* **시각화**: 3차원 이상은 시각화가 어려운데, 차원 축소를 통해 시각화하기 용이해집니다.
* **노이즈(잡음) 제거**: 데이터의 불필요한 노이즈를 줄이는 데 도움을 줍니다.
* **과적합 방지**: 특성이 너무 많으면 과적합이 일어나기 쉬운데, PCA는 이를 방지하여 모델의 일반화에 기여합니다.
* **계산 속도 향상**: 특성의 수가 줄어들어 모델 학습 및 예측의 계산 속도가 빨라집니다.

---
## 1. 라이브러리 임포트

필요한 라이브러리들을 임포트합니다.

In [ ]:
from sklearn.datasets import load_breast_cancer, load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt # 이전 버전에서는 pyplot가 반드시 먼저 import가 되어야 한다.
import seaborn as sns # seaborn도 pyplot 기반이라서
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

---
## 2. 데이터 로드 및 확인

`load_digits` 데이터셋을 로드하고 특성 데이터(`X`)와 타겟 데이터(`y`)의 형태를 확인합니다.

In [ ]:
# cancer = load_breast_cancer() # 유방암 데이터셋 (주석 처리)
cancer = load_digits() # 숫자 손글씨 데이터셋
X = cancer['data']
y = cancer['target']

print("원본 데이터의 형태 (X.shape):", X.shape)

---
## 3. 데이터 스케일링

PCA를 적용하기 전에 데이터를 표준 스케일링(`StandardScaler`)합니다. 이는 각 특성의 스케일이 PCA 결과에 미치는 영향을 균일하게 하기 위함입니다.

In [ ]:
scalar = StandardScaler()
scalar.fit(X)
X_scaled = scalar.transform(X)

print("스케일링된 데이터의 형태 (X_scaled.shape):", X_scaled.shape)

---
## 4. PCA(주성분분석) 적용

스케일링된 데이터에 PCA를 적용하여 차원을 축소합니다. 여기서는 10개의 주성분을 추출합니다.

In [ ]:
pca = PCA(n_components=10) # 성분 개수 지정하기
pca.fit(X_scaled) # 학습
X_pca = pca.transform(X_scaled)

print("PCA 변환된 데이터의 형태 (X_pca.shape):", X_pca.shape)

---
## 5. 데이터 분할

원본 데이터, 스케일링된 데이터, PCA 변환된 데이터를 각각 훈련 세트와 테스트 세트로 분할합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y, random_state=0)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, random_state=0)

---
## 6. 로지스틱 회귀 모델 학습 및 평가

각 데이터셋(원본, 스케일링, PCA)에 대해 로지스틱 회귀 모델을 학습하고 훈련 및 테스트 세트에서의 정확도를 비교합니다.

In [ ]:
model = LogisticRegression(max_iter=10000) # 수렴 경고를 피하기 위해 max_iter 증가
model.fit(X_train, y_train)
print("-------- 기본 데이터 --------- ")
print("훈련셋 정확도:", model.score(X_train, y_train))
print("테스트셋 정확도:", model.score(X_test, y_test))

model.fit(X_train_scaled, y_train_scaled)
print("\n-------- 스케일링된 데이터 --------- ")
print("훈련셋 정확도:", model.score(X_train_scaled, y_train_scaled))
print("테스트셋 정확도:", model.score(X_test_scaled, y_test_scaled))

model.fit(X_train_pca, y_train_pca)
print("\n-------- PCA 변환된 데이터 --------- ")
print("훈련셋 정확도:", model.score(X_train_pca, y_train_pca))
print("테스트셋 정확도:", model.score(X_test_pca, y_test_pca))